<https://realpython.com/python-concurrency/>

___What Is Concurrency?___


__Threading and asyncio__ both run on a single processor and therefore only run one at a time. They just cleverly find ways to take turns to speed up the overall process. Even though they don’t run different trains of thought simultaneously, we still call this concurrency.


 In __threading__, the operating system actually knows about each thread and can interrupt it at any time to start running a different thread. This is called __pre-emptive multitasking__ since the operating system can pre-empt your thread to make the switch.
 
 __Asyncio__, on the other hand, uses __cooperative multitasking__. The tasks must cooperate by announcing when they are ready to be switched out. That means that the code in the task has to change slightly to make this happen.


__What Is Parallelism?__

With __multiprocessing__, Python creates new __processes__. A process here can be thought of as almost a completely different program, though technically they’re usually defined as a collection of resources where the resources include memory, file handles and things like that. One way to think about it is that each process runs in its own Python interpreter.



__Synchronous Version__

In [6]:
import requests
import time


def download_site(url, session):
    st = time.time()
    with session.get(url) as response:
        dura = time.time() - st
        print(f"Read {len(response.content)} from {url},time taken: {dura:.3f} sec")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 30
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10782 from https://www.jython.org,time taken: 0.312 sec
Read 277 from http://olympus.realpython.org/dice,time taken: 0.638 sec
Read 10782 from https://www.jython.org,time taken: 0.099 sec
Read 277 from http://olympus.realpython.org/dice,time taken: 0.269 sec
Read 10782 from https://www.jython.org,time taken: 0.107 sec
Read 277 from http://olympus.realpython.org/dice,time taken: 1.050 sec
Read 10782 from https://www.jython.org,time taken: 0.100 sec
Read 277 from http://olympus.realpython.org/dice,time taken: 0.269 sec
Read 10782 from https://www.jython.org,time taken: 0.101 sec
Read 277 from http://olympus.realpython.org/dice,time taken: 0.273 sec
Read 10782 from https://www.jython.org,time taken: 0.101 sec
Read 277 from http://olympus.realpython.org/dice,time taken: 0.269 sec
Read 10782 from https://www.jython.org,time taken: 0.100 sec
Read 277 from http://olympus.realpython.org/dice,time taken: 0.270 sec
Read 10782 from https://www.jython.org,time taken: 0.100 sec
Read 277 from h

__threading Version__

Executor maps download_site() function call to a thread only after the current train of thought gets completed.

Session is not thread-safe, we use seperate session object for each thread.

In [5]:
import concurrent.futures
import requests
import threading
import time


thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    print(f'{thread_local.session}')
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}, thread: {thread_local.session}")

def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 30
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E6E8EB0>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E454F10>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E6EA680>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E454F10>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E454F10>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E65FC70>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E6E9330>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E65FC70>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E6E9330>
Read 277 from http://olym

Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E6EA680>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E65FC70>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E454F10>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E6EA680>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E454F10>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E6E9330>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E6E8EB0>
Read 10782 from https://www.jython.org, thread: <requests.sessions.Session object at 0x000002959E454F10>
Read 277 from http://olympus.realpython.org/dice, thread: <requests.sessions.Session object at 0x000002959E65FC70>
Read 

__asyncio Version__

jupyter environ. spits error for asyncio code